### This is the latest version of a small baseline model. Due to unfamiliarity with signal analysis, utilization of SciPy's signal functions may be incorrect.

### To-Do List
* In-Depth EDA
* Optimization of dataset generation. Currently going from Pandas to non pandas to Pandas is not efficient.
* Compare model output with feature selection criteria.
* Hyperparameter Tuning. PyCaret allows for AutoTuning but i've had a few issues with this dataset, so i will be tuning the winning model from PyCaret with the usual GridSearchCV from sklearn.


Library loading

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np # linear algebra
import pandas as pd
from scipy import signal
from scipy import signal
from scipy.signal import find_peaks
!pip install pycaret
from pycaret.regression import *

### Training set generation:

Currently approach is to extract a few features from each of the time-series(TS) in each of the .csv files to generate a single row in a final dataset that will correspond to the id and its matching 'time_to_eruption'. Null values are imputted as 0. This is not a correct approach and is done for the baseline procedure, will change inputting strategy once EDA is perfomed.

Current Features:
* Basic Stats(mean,std,median,max,etc.) of original TS, output of df.describe without 'count'.
* Basic Stats of periodogram of original TS.
* Amount of Stds that the Max or Min amount to
* Number of peaks in each sensor.
* Basic Stats(mean,std) of the prominence of the peaks. 

In [ ]:
from pathlib import Path
import time
start = time.time()
A = []
rownames = []
pathlist = sorted(Path('../input/predict-volcanic-eruptions-ingv-oe/train/').glob('**/*.csv'))
c=0
for path in pathlist:
    path_in_str = str(path)
    dat=pd.read_csv(path_in_str)
    dat=dat.fillna(0) #Handling NaN in this manner is not correct. But this is a baseline.
    dat2=dat.describe().iloc[1:,:]
    dat2=pd.concat([dat2,pd.DataFrame(signal.periodogram(dat)[1]).describe().iloc[1:,:]],axis=1) 
    dat2=pd.DataFrame(dat2.unstack()).T
    for (columnName, columnData) in dat.iteritems():
        dat2[str(columnName)+ '_Mode'] = dat[str(columnName)].mode()
        dat2[str(columnName)+ '_MaxStds'] = [dat[str(columnName)].max()/dat[str(columnName)].std() if dat[str(columnName)].std()!=0 else np.nan]
        dat2[str(columnName)+ '_MinStds'] = [dat[str(columnName)].min()/dat[str(columnName)].std() if dat[str(columnName)].std()!=0 else np.nan]
        dat2[str(columnName)+ '_PeakNumber']=len(find_peaks(dat[str(columnName)])[0])
        dat2[str(columnName)+ '_MeanProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].mean()
        dat2[str(columnName)+ '_StdProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].std()
        #dat2[str(columnName)+ '_MaxProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].max()
        #dat2[str(columnName)+ '_MinProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].min()
    dat2.index=dat2.index.to_flat_index()
    A.append(dat2.T.values.flatten())
    rownames.append(path.stem)
    c=c+1
    print(path.stem,c)
print(time.time()-start)

### Test set generation:

Same as Training set description.

In [ ]:
from pathlib import Path
A_test = []
rownames_test = []
c=0
pathlist = sorted(Path('../input/predict-volcanic-eruptions-ingv-oe/test/').glob('**/*.csv'),reverse=True)
for path in pathlist:
     # because path is object not string
    path_in_str = str(path)
    dat=pd.read_csv(path_in_str)
    dat=dat.fillna(dat.mean(0))
    dat2=dat.describe().iloc[1:,:]
    dat2=pd.concat([dat2,pd.DataFrame(signal.periodogram(dat)[1]).describe().iloc[1:,:]],axis=1)
    dat2=pd.DataFrame(dat2.unstack()).T
    for (columnName, columnData) in dat.iteritems():
        dat2[str(columnName)+ '_Mode'] = dat[str(columnName)].mode()
        dat2[str(columnName)+ 'MaxStds'] = [dat[str(columnName)].max()/dat[str(columnName)].std() if dat[str(columnName)].std()!=0 else np.nan]
        dat2[str(columnName)+ 'MinStds'] = [dat[str(columnName)].min()/dat[str(columnName)].std() if dat[str(columnName)].std()!=0 else np.nan]
        dat2[str(columnName)+ '_PeakNumber']=len(find_peaks(dat[str(columnName)])[0])
        dat2[str(columnName)+ '_MeanProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].mean()
        dat2[str(columnName)+ '_StdProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].std()
        #dat2[str(columnName)+ '_MaxProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].max()
        #dat2[str(columnName)+ '_MinProminence']=signal.peak_prominences(dat[str(columnName)],signal.find_peaks(dat[str(columnName)])[0])[0].min()
    dat2.index=dat2.index.to_flat_index()
    A_test.append(dat2.T.values.flatten())
    rownames_test.append(path.stem)
    c=c+1
    print(path.stem,c)

#### Tidying up the output from the loops.

In [ ]:
A = pd.DataFrame(A)
A['segment_id']=rownames
A.index=A['segment_id']
A=A.drop('segment_id',axis=1)
A.index=A.index.astype('int32')
target = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train.csv')
target.index=target['segment_id']
target=target.drop('segment_id',axis=1)
B=pd.merge(A, target, left_index=True, right_index=True)
B=B.astype('float32')

### Feature Selection:

Basic feature selection with sklearn to lower amount of features. 
Although, 190 features is not that many considering the size of the dataset, but lower features returned better results this time.
This should be optimized by quick checking the effect of k-features on model MAE.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression
B=B.fillna(B.mean())
Selection = SelectKBest(mutual_info_regression, k=80).fit(B.iloc[:,:-1], B['time_to_eruption'])
X_new=Selection.transform(B.iloc[:,:-1])
X_new1=pd.DataFrame(X_new)
X_new1.index=B.index
X_new1['time_to_eruption']=B['time_to_eruption']
X_new1

### Model Selection:

The AutoML library PyCaret will be employed for this approach.
It will perform the training over many supervised models and return a mean of a 10-Fold for each of them. 
(Hyperparameters are *not* modified in this part).

In [ ]:
comparison_setup = setup(X_new1, target = 'time_to_eruption',session_id=17,silent = True)
compare_models()

The best model is ExtraTrees, so i will create a new training with just this model and finalize it to perform predictions over the test dataset.

In [ ]:
xt_clu = setup(X_new1, target = 'time_to_eruption',session_id=17,silent = True)
et1 = create_model('et')
final_et1 = finalize_model(et1)

### Tidying up the test database

In [ ]:
A_test=pd.DataFrame(A_test)
A_test['id']=rownames_test
A_test.index=A_test['id'].astype('int32')
A_test=A_test.drop(['id'],axis=1) 
A_test2=A_test.fillna(0)

### Apply the feature selection to test

In [ ]:
X_test=Selection.transform(A_test2)
X_test=pd.DataFrame(X_test)
X_test.index=rownames_test

### Predictions:

In [ ]:
predictions = predict_model(final_et1, data = X_test)
sub=predictions[['Label']]
sub['segment_id']=sub.index
sub['time_to_eruption']=predictions['Label']
sub=sub.drop('Label',axis=1)
sub

### Create submission for competition:

In [ ]:
sub.to_csv('submission.csv',index=False)